In [4]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)


Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:705)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:749)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:673)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:639)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:615)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:572)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:530)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [6]:
tbl_merchants = spark.read.parquet("../data/tables/tbl_merchants.parquet")

In [8]:
from pyspark.sql.functions import *
tbl_merchants = tbl_merchants.toPandas()

In [48]:
# Observe strings format
tbl_merchants["tags"][0]

'((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))'

In [49]:
# Separate tags from revenue and take rate
tags1 = tbl_merchants["tags"].str.split("\), ", expand=True)
tags2 = tags[0].str.split("\], ", expand=True)

In [65]:
# Remove symbols from tag and making everything lowercase
tags3 = tags2[0].str.replace('[^\w\s]', '', regex = True)
tags4 = tags3.str.lower()

In [66]:
tags4

0       furniture home furnishings and equipment shops...
1       cable satellite and other pay television and r...
2                jewelry watch clock and silverware shops
3                    watch clock and jewelry repair shops
4       music shops  musical instruments pianos and sh...
                              ...                        
4021               opticians optical goods and eyeglasses
4022                     books periodicals and newspapers
4023                                          shoe  shops
4024                 motor vehicle supplies and new parts
4025                 motor vehicle supplies and new parts
Name: 0, Length: 4026, dtype: object

In [67]:
# https://www.shopify.com/au/blog/top-online-shopping-categories
categories = ["fashion", "electronics", "toys, hobby and DIY", "furniture", 
              "beauty, health, personal and household care", "other"]

In [63]:
for i in range(0, len(tags4)):
    for category in categories:
        if category in tags4[i]:
            tags4[i] = category

In [68]:
tags4

0       furniture home furnishings and equipment shops...
1       cable satellite and other pay television and r...
2                jewelry watch clock and silverware shops
3                    watch clock and jewelry repair shops
4       music shops  musical instruments pianos and sh...
                              ...                        
4021               opticians optical goods and eyeglasses
4022                     books periodicals and newspapers
4023                                          shoe  shops
4024                 motor vehicle supplies and new parts
4025                 motor vehicle supplies and new parts
Name: 0, Length: 4026, dtype: object